# Point in Polygon
## Damien Allison, Drew Mattson, Caleb Andreano

### Computational Problem:
Given a series of two-dimensional points whose composition creates a simple polygon, determine if a distinct point is within the polygon.

### Decision Rule:
The decision rule used to determine whether a given point is the following: given a line from the target point parallel to the $x$ axis in the positive $x$ direction, if the number of intersections between the line and the edges of the polgon as defined by the given vertices is odd, the point is in the polygon. Otherwise, the point is not in the polygon. This decision rule is augmented by the following clauses:
- If the target point is outside of the range of the given vertices (if it is not in between the minimum and maximum $y$ bounds), then the point is outside the polygon.
- If the $x$ position of the point is not in the domain of the given verticies (if it is not between the minumum and maximum $x$ bounds), the point is outside the polygon.  

For example: 
- Given polygon is `[(1, 1), (1, 3), (3, 3), (3, 1)]`. Target point is `(2, 2)`. The $y$ range is $(1, 3)$ and the minumum $x$ is $1$. Since the point is not outside of these bounds, continue. Casting a ray to the right from `(2, 2)`, we see one intersection. So, the point is in the polygon.
- Given polygon is `[(1, 1), (1, 3), (3, 3), (3, 1)]`. Target point is `(4, 2)`. The $y$ range is $(1, 3)$ and the minimum $x$ is $1$. Since the point is not outside of these bounds, continue. Casting a ray to the right from `(4, 2)`, we see zero intersections. So, the point is not in the polygon.

### Pseudocode:
```
let max_x = max(vertice1.x, vertice2.x)
let min_y, max_y = min(vertice1.x, vertice2.x), max(vertice1.x, vertice2.x)

if point.x < max_x and min_y < point.y < max_y do:
    num_intersections = 0;
    for (vertice1, vertice2) in vertices.pairs() do:
        if intersection(point.y, vertice1, vertice2):
            num_intersections += 1
        end if
    end for

    return num_intersections % 2 == 1;
else do:
    return false;
end if
```

### Justification:

The algorithm relies on the principle that when casting a horizontal ray from the point, if the ray intersects the polygon an odd number of times, it is inside the polygon, and if does not intersect or it intersects and even number of times, the point is outside the polygon. While this is an intuitive principle, the proof is not trivial and requires homology theory to generalize and show. However, this principle is proven by the Jordan curve theorem, which states:

"If the initial point $p_a$ of a ray lies outside a simple polygon $A$, the number of intersections of the ray and the polygon is even. If the inital point $p_b$ of a ray lies outside the polygon, the number of intersections is odd". (Wikipedia)

Our algorithm correctly implements this principle, as it computes the intersection between the horizontal ray with the target point as the origin and the intersection between the edge created by each pair of vertices using the following formula:

$$
i_x = \frac{(y-y_0)(x_1 - x_0) - x_0(y_1-y_0)}{y_0 - y_1}
$$
where
- $i_x$ is the $x$ coordinate of the intercept,
- $x,~y$ are the $x$ and $y$ coordinates of the target point,
- $x_0,~y_0$ are the coordinates of the first vertice
- $y_0,~y_1$ are the coordinates of the second vertice

The algorithm is guaranteed to halt because the loop iterates through all pairs. Since there is a finite number of pairs, the loop will stop once every pair has been used. 

In [10]:
from matplotlib import pyplot as plt


def plot(poly, point):
    x, y = point
    fig, ax = plt.subplots()
    ax.set_xlim(min(poly, key=lambda p: p[0])[0] - 1, max(poly, key=lambda p: p[0])[0] + 1)
    ax.set_ylim(min(poly, key=lambda p: p[1])[1] - 1, max(poly, key=lambda p: p[1])[1] + 1)
    ax.add_patch(Polygon(poly, closed=True, fill=False, color='g', linewidth=2))
    plt.plot(x, y, 'ro', label=f'{x}, {y}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'Polygon with point ({x}, {y})')
    plt.legend()
    plt.grid(True)
    plt.show()

In [1]:
import math


# The intersection function returns the point of intersection between the
# line that is made from p1 and p0 and the horizontal line y.
def intersection(p1, p0, y):
    return math.nan \
        if p1[1] - p0[1] == 0 \
        else ((y - p0[1]) * (p1[0] - p0[0]) - p0[0] * (p1[1] - p0[1])) / (p0[1] - p1[1])


# The in_bounds function returns true if the x value of the intersection is
# within the min, being the target x, and max bounds.
def in_bounds(target_x, intersect_x, max_x):
    return not math.isnan(intersect_x) and target_x is not max_x and target_x <= intersect_x <= max_x


# The in_polygon function determines if a point is inside a polygon made of other points.
def in_polygon(points, target):
    # Finds the bounds that the intersections must be made in.
    max_y = min_y = points[0][1]
    max_x = points[0][0]
    for x, y in points:
        min_y, max_y, max_x = min(min_y, y), max(max_y, y), max(max_x, x)

    # If the target point is beyond the bounds of the polygon (above or below),
    # the point is certainly not in the polygon
    return (min_y < target[1] < max_y and
    # A point is within the polygon if the number of intersections
    # within the bounds is an odd number. This is accomplished by iterating
    # through the points list and adding one to a sum if the intersection occurs
    # within the bounds.
            sum(1 for i in range(len(points)) if in_bounds(target[0],intersection(points[i], points[(i + 1) % len(points)], target[1]),
                max_x
            )) % 2 == 1)

### Asymptotic Analysis of the worst-case run-time of your algorithm

The worst-case run time occurs when the distinct point is within the vertical bounds of the polygon. When this occurs, not only are the bounds of the polygon are found, but the algorithm must also calculate the intersections of each line with y = Y, where Y is the y value of the target. This is done in Θ(2n) time or simply Θ(n) time.

### Test Cases:
The input domain can be partitioned in a few ways which can be more easily seen in graph from below:
<brk/>
<img src="./Test Case Analysis.png" width="500">
There are 5 partitions within the input domain. These are: 
 - Green: Above the maximum y value of the polygon
 - Red: Left of the minimum x value of the polygon
 - White: Inside the polygon
 - Blue: Right of the maximum x value of the polygon
 - Purple: Below the minimum y value of the polygon

There are also 8 points on the boundaries of the partitions which also must be tested. These can be seen as the black points in the image above.

All test cases provide the distinct point and are all tested against a square for convenience.
The points that make up the square are: (2, 2), (2, 4), (4,4), (4, 2)

| Title                                   | Description | Expected Answer | Actual Answer |
|:----------------------------------------|:------------|:----------------|:--------------|
| In the Polygon                          | (3, 3)      | True            | True          |
| Above the Polygon                       | (3, 5)      | False           | False         |
| Left of the Polygon                     | (1, 3)      | False           | False         |
| Right of the Polygon                    | (5, 3)      | False           | False         |
| Below the Polygon                       | (3, 1)      | False           | False         |
| On top boundary                         | (3, 4)      | False           | False         |
| On left boundary                        | (2, 3)      | False           | False         |
| On right boundary                       | (4, 3)      | False           | False         |
| On bottom boundary                      | (3, 2)      | False           | False         |
| In line & left of <br/>bottom boundary  | (1, 2)      | False           | False         |
| In line & right of <br/>bottom boundary | (5, 2)      | False           | False         |
| In line & left of <br/>top boundary     | (1, 4)      | False           | False         |
| In line & right of <br/>top boundary    | (5, 4)      | False           | False         |



In [12]:
test_polygon = [(2, 2), (2, 4), (4, 4), (4, 2)]
test_cases = [
    {"Point": (3, 3), "Expected": True},  # In the polygon

    {"Point": (3, 5), "Expected": False},  # Above the polygon
    {"Point": (1, 3), "Expected": False},  # Left of the polygon
    {"Point": (5, 3), "Expected": False},  # Right of the polygon
    {"Point": (3, 1), "Expected": False},  # Below the polygon

    {"Point": (3, 4), "Expected": False},  # On top boundary
    {"Point": (2, 3), "Expected": False},  # On left boundary
    {"Point": (4, 3), "Expected": False},  # On right boundary
    {"Point": (3, 2), "Expected": False},  # On bottom boundary

    {"Point": (1, 2), "Expected": False},  # In line & left of bottom boundary
    {"Point": (5, 2), "Expected": False},  # In line & right of bottom boundary
    {"Point": (1, 4), "Expected": False},  # In line & left of top boundary
    {"Point": (5, 4), "Expected": False},  # In line & right of top boundary
]

failed_tests = list(filter(
    lambda i: test_cases[i]["Expected"] is not in_polygon(test_polygon, test_cases[i]["Point"]),
    range(len(test_cases))
))

passed = len(test_cases) - len(failed_tests)
print("Passed:", str(passed) + "/" + str(len(test_cases)))

if failed_tests:
    print("(X, Y)\t->\tExpected\t|\tActual")
    print("-----------------------------------")

    for t in failed_tests:
        point = test_cases[t]["Point"]
        expected = test_cases[t]["Expected"]
        actual = in_polygon(test_polygon, point)
        print(f"{point}\t->\t{expected}\t\t|\t{actual}")


Passed: 13/13


### Benchmarking:
<Placeholder: A table and graph from benchmarking your implementation on problem instances of different sizes

### Theory vs Actuality
<Placeholder: A paragraph comparing your benchmarking results to your theoretical asymptotic run-time.> 
<Placeholder: This should include an explanation of whether your results support your theoretically-derived run-time.>